In [2]:
import copy
#from base_test import BaseTest
#import digits_model
import numpy as np
import matplotlib.pyplot as plt
#import data

import torch
import torchvision
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torchvision.transforms as transforms
import time
import os
#from data import NormalizeRangeTanh, UnNormalizeRangeTanh - What the hell is this?

In [4]:
class generator(nn.Module):

    def __init__(self, channels, gpu=False): # What to do with GPU?
        super(self.__class__,self).__init__()
        self.channels=channels
        self.gpu=gpu
        self.network=nn.Sequential(
                     nn.ConvTranspose2d(self.channels,512,kernel_size=(4,4), stride=1, padding=1),
                     nn.BatchNorm1d(512),
                     nn.ReLU(inplace=True),
                     nn.ConvTranspose2d(512,256,kernel_size=(4,4), stride=2, padding=1),
                     nn.BatchNorm1d(256),
                     nn.ReLU(inplace=True),
                     nn.ConvTranspose2d(256,128,kernel_size=(4,4), stride=2, padding=1),
                     nn.BatchNorm1d(128),
                     nn.ReLU(inplace=True),
                     nn.ConvTranspose2d(128,1,kernel_size=(4,4), stride=2, padding=1),
                     nn.Tanh()
        )
    def forward(self,input):
        return self.network(input)

In [5]:
class discriminator(nn.Module):
  
    def __init__(self, channels, leaky_coef=0.2):
        super(self.__class__,self).__init__()
        self.channels=channels
        self.leaky_coef=leaky_coef
        self.network=nn.Sequential(
                     nn.Conv2d(1,128,kernel_size=(3,3), stride=2, padding=1),
                     nn.BatchNorm2d(128),
                     nn.LeakyReLU(self.leaky_coef,inplace=True),
                     nn.Conv2d(128,256,kernel_size=(3,3), stride=2, padding=1),
                     nn.BatchNorm2d(256),
                     nn.LeakyReLU(self.leaky_coef,inplace=True),
                     nn.Conv2d(256,512,kernel_size=(3,3), stride=2, padding=1),
                     nn.BatchNorm2d(512),
                     nn.LeakyReLU(self.leaky_coef,inplace=True),
                     nn.Conv2d(512,3, kernel_size=(4,4), stride=2)
        )

    def forward(self,input):
        return self.network(input)

In [18]:
class Reshape(nn.Module):
    def __init__(self, *args):
        super(Reshape, self).__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)

class feature_encoder(nn.Module):
    def __init__(self, channels, gpu=False):
        super(feature_encoder, self).__init__()
        self.gpu = gpu
        self.channels = channels
        self.classify = nn.Sequential(
                       nn.Conv2d(3, 64, kernel_size=3, padding=1),
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(2),               
                    nn.Conv2d(64, 128, kernel_size=3, padding=1),
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(2),
                    nn.Conv2d(128, 256, kernel_size=3, padding=1),
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(2),
                    nn.Conv2d(256, 128, kernel_size=4, padding=0))
#                     nn.ReLU(inplace=True),
#                     nn.MaxPool2d(4),
#                     Reshape(-1,128),
#                     nn.Linear(128, 10),
#                     nn.Softmax(),)
        if self.gpu:
            self.type(torch.cuda.FloatTensor)

    def forward(self, input):
        return self.classify(input)

In [46]:
# def classifier(address):
#    # Kagi will guide us
#   pass

class tanh_normalize(object):
    ''' Normalizes a tensor with values from [0, 1] to [-1, 1]. '''
    def __init__(self):
        pass
    
    def __call__(self, sample):
        sample = sample * 2.0 - 1.0
        return sample

class digits_transfer():
  
  
    def __init__(self,gpu=True):
    
        super(digits_transfer,self).__init__()
        self.gpu = gpu
        self.generator_loss_function=None
        self.gan_loss_function=None
        self.discriminator_loss_function=None
        #self.source_validation_loader=None
        self.source_test_loader=None
        self.target_test_loader=None
        self.distance_target_domain=None
        self.source_train_loader=None
        self.target_train_loader=None
        self.batch_size=128
        self.lossCrossEntropy=nn.CrossEntropyLoss() ## Why the fuck is it here?
    
    def readClassifier(self, model_name):

    #     old_model = torch.load(model_name)['state_dict']
    #     old_dict = old_model.load_state_dict() 
    #     new_model = feature_encoder(3,self.gpu)
    #     new_dict = new_model.state_dict()
    #     new_dict = {k: v for k, v in old_dict.items() if k in new_dict}
    #     old_dict.update(new_dict) 
    #     new_model.load_state_dict(new_dict)

    #     model = describe_model()
    #     checkpoint = torch.load('checkpoint.pth.tar')
    #     model.load_state_dict(checkpoint['state_dict'])

        model = feature_encoder(3,self.gpu)
        temp = torch.load(model_name)
        model.load_state_dict(temp['state_dict'])
        self.model['f'] =model

        for param in self.model['f'].parameters():
            param.requires_grad = False
    
    def data_loader(self):

        SVHN = transforms.Compose([transforms.ToTensor(), tanh_normalize()])
        MNIST = transforms.Compose([transforms.Scale(32), transforms.ToTensor(), tanh_normalize()])

        source_train_set = torchvision.datasets.SVHN(root='./SVHN/', split='extra', download=True, transform=SVHN)
        self.source_train_loader = torch.utils.data.DataLoader(source_train_set, batch_size=128, shuffle=True, num_workers=8)

        source_test_set = torchvision.datasets.SVHN(root='./SVHN/', split='test', download=True, transform=SVHN)
        self.source_test_loader = torch.utils.data.DataLoader(source_test_set, batch_size=128, shuffle=False, num_workers=8)

        target_train_set = torchvision.datasets.MNIST(root='./MNIST/', train=True, download=True, transform=MNIST)
        self.target_train_loader = torch.utils.data.DataLoader(target_train_set, batch_size=128, shuffle=True, num_workers=8)

        target_test_set = torchvision.datasets.MNIST(root='./MNIST/', train=False, download=True, transform=MNIST)
        self.target_test_loader = torch.utils.data.DataLoader(target_test_set, batch_size=128, shuffle=False, num_workers=8)

    def view_batch(self):
    ## TO be filled in later
        pass
  
    def make_model(self):

        self.model = {}
        self.model['d'] = discriminator(128)
        self.model['g'] = generator(128)
        if self.gpu:
          self.model['d'] = self.model['d'].cuda()
          self.model['g'] = self.model['g'].cuda()
        # Assumption 1 : Kagi saves everything as a Tar File
        #self.model['f'] = classifier('Give the saved model here')
        self.readClassifier('final_f.tar')

    def create_discriminator_loss_function(self):

        def discriminator_loss(s_d_g, t_d_g, t_d):  
            return self.lossCrossEntropy(s_d_g.squeeze(), self.label_0) + self.lossCrossEntropy(t_d_g, self.label_1) + self.lossCrossEntropy(t_d, self.label_2)

        self.discriminator_loss_function = discriminator_loss
  
    def create_generator_loss_function(self):

        def generator_loss(s_f, s_g_f, s_d_g, t, t_g, t_d_g, alpha, beta, gamma):

            loss_gan_generator = self.lossCrossEntropy(s_d_g.squeeze, self.label_2) + self.lossCrossEntropy(t_d_g, self.label_2)
            loss_feature_constancy = self.mean_square_loss(s_g_f, s_f.detach())
            loss_target_identity = self.mean_square_loss(t_g, t.detach())

            return loss_gan_generator + alpha*loss_feature_constancy + beta*loss_target_identity

        self.generator_loss_function = generator_loss
  

    def make_loss_function(self):
        self.lossCrossEntropy = nn.CrossEntropyLoss().cuda()
        self.mean_square_loss = nn.MSELoss().cuda()
        label_0, label_1, label_2 = (torch.LongTensor(self.batch_size) for i in range(3))
        label_0 = Variable(label_0.cuda())
        label_1 = Variable(label_1.cuda())
        label_2 = Variable(label_2.cuda())
        label_0.data.resize_(self.batch_size).fill_(0)
        label_1.data.resize_(self.batch_size).fill_(1)
        label_2.data.resize_(self.batch_size).fill_(2)
        self.label_0 = label_0
        self.label_1 = label_1
        self.label_2 = label_2

    #     self.create_distance_function_target_domain()
        self.create_discriminator_loss_function()
        self.create_generator_loss_function()
  
    def make_optimizer(self):
    
        generator_learning_rate = 0.001
        generator_weight_decay = 0.000001     
        self.generator_optimizer = optim.Adam(self.model['g'].parameters(), lr = generator_learning_rate, weight_decay = generator_weight_decay)

        discriminator_learning_rate = 0.001
        discriminator_weight_decay = 0.000001     
        self.discriminator_optimizer = optim.Adam(self.model['d'].parameters(), lr = discriminator_learning_rate, weight_decay = discriminator_weight_decay)

    #   def validate_model(self, **kwargs):

    #     gan_loss_weight = kwargs.get("gan_loss_weight",0.001)
    #     validation_loss = 0
    #     self.model['g'].eval()
    #     samples = np.random.randint(0,len(source_validation_set),size = 5)
    #     for i in samples:
    # #       ## source_validation_set is a keyword argument!
    # #       source_datLLLa = source_validation_set[i]
    # #       source_generator_output = self.model['g'](source_data)
    # #       source_classifier_output = self.model['f'](source_data)
    # #       source_
    # #     """
    # #       To be Done Later
    # #     """
    # #     pass

    def show_image():
        pass
        # Fuck this shit
    def display_results(self, source_data, source_generator):
        """
          To be done later"""
        pass
    
    def model_train(self, number_of_epochs, **kwargs):
    
        visualize_batch = kwargs.get("visualize_batch",50)
        save_batch = kwargs.get("save_batch",200)
        test_batch = kwargs.get("test_batch",200)

        SVHN_count=0
        batch_count=0
        discriminator_run_loss=0
        generator_run_loss=0

        l=min(len(self.source_train_loader),len(self.target_train_loader))

        for epoch in range(number_of_epochs):

            source_data_iter=iter(self.source_train_loader)
            target_data_iter=iter(self.target_train_loader)

            for i in range(l):

                SVHN_count+=1
                if SVHN_count>=len(self.source_train_loader):
                    SVHN_count=0
                    source_data_iter=iter(self.source_train_loader)

                source_data, source_labels=source_data_iter.next()
                target_data, target_labels=source_data_iter.next()

                if self.batch_size != source_data.size(0) or self.batch_size != target_data.size(0):
                    continue

                batch_count+=1

                if self.gpu:
                    source_data, source_labels = Variable(source_data.float().cuda()), Variable(source_labels.float().cuda())
                    target_data, target_labels = Variable(target_data.float().cuda()), Variable(target_labels.float().cuda())

                else:
                    source_data, source_labels = Variable(source_data.float()), Variable(source_labels.float())
                    target_data, target_labels = Variable(target_data.float()), Variable(target_labels.float())

                for param in self.model['d'].parameters():
                    param.requires_grad = True

                self.model['d'].zero_grad()

                s_f = self.model['f'](source_data)      #generator 
                print(s_f.shape)
                s_g = self.model['g'](s_f)
                s_g_detach = s_g.detach()
                s_d_g = self.model['d'](s_g_detach)

                t_data_3 = torch.cat((t_data,t_data,t_data),1)
                t_f = self.model['f'](t_data_3)
                t_g = self.model['g'](t_f)
                t_g_detach = t_g.detach()
                t_d_g = self.model['d'](t_g_detach)
                t_d = self.model['d'](t_data)

                Discriminator_loss = self.discriminator_loss_function(s_d_g, t_d_g, t_d)
                Discriminator_loss.backward()
                self.discriminator_optimizer.step()

                for param in self.model['g'].parameters():
                      param.requires_grad = False

                self.model['g'].zero_grad()

                s_d_g = self.model['d'](s_g)
                s_g_3 = torch.cat((s_g,s_g,s_g),1)
                s_g_f = self.model['f'](s_g_3)

                t_d_g = self.model['d'](t_g)

                Generator_loss = self.generator_loss_function(s_f, s_g_f, s_d_g, t, t_g, t_d_g)
                Generator_loss.backward()
                self.generator_optimizer.step()

                discriminator_run_loss+=Discriminator_loss.data[0]
                generator_run_loss+=Generator_loss.data[0]

In [48]:
dig=digits_transfer()

In [49]:
dig.make_model()

In [22]:
m = torch.load('f_model.tar')

In [37]:
from collections import OrderedDict
new_dict = OrderedDict()
final_dict = {'state_dict':new_dict}

for elem in m['state_dict']:
    #print(elem,(m['state_dict'][elem].shape))
    if '13' not in elem:
        final_dict['state_dict'][elem] = m['state_dict'][elem]

In [39]:
torch.save(final_dict, 'final_f.tar')

In [34]:
print(type(m['state_dict']))a

<class 'collections.OrderedDict'>


In [50]:
dig.data_loader()

Using downloaded and verified file: ./SVHN/extra_32x32.mat
Using downloaded and verified file: ./SVHN/test_32x32.mat


In [51]:
dig.make_optimizer()
dig.make_loss_function()

In [52]:
dig.model_train(1)

torch.Size([128, 128, 1, 1])


ValueError: expected 2D or 3D input (got 4D input)

Object `torch.view` not found.
